<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%9C%D1%91%D1%80%D0%B4%D0%B6%20%D1%81%20TF-IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Загружаем файл с дополнительной информацией (столбцы D:AC)
extra_data = pd.read_excel("лексика мигранты однословные (определение типа лексемы).xlsx", usecols="D:AC")

# Загружаем основной CSV-файл
main_data = pd.read_csv("обновленная_лексика_мигранты.csv")

# Проверим, совпадают ли размеры
if len(main_data) != len(extra_data):
    print(f"⚠️ Внимание: количество строк в файлах отличается! main_data: {len(main_data)}, extra_data: {len(extra_data)}")
else:
    print(f"✅ Строки совпадают: {len(main_data)}")

# Объединяем по индексу
merged_data = pd.concat([main_data, extra_data], axis=1)

# Сохраняем результат
merged_data.to_csv("обновленная_лексика_мигранты_с_типами.csv", index=False)

print("Файл обновленная_лексика_мигранты_с_типами.csv сохранён.")

✅ Строки совпадают: 10274
Файл обновленная_лексика_мигранты_с_типами.csv сохранён.


In [2]:
import pandas as pd

# Загружаем файл
df = pd.read_csv("обновленная_лексика_мигранты_с_типами.csv")

# Создаём новый столбец "words", извлекая первую часть перед "__"
df["words"] = df["lemma__pos"].str.split("__").str[0]

# Сохраняем обновлённый файл
df.to_csv("обновленная_лексика_мигранты_обработанная.csv", index=False)

print("Файл обновленная_лексика_мигранты_обработанная.csv сохранён.")

Файл обновленная_лексика_мигранты_обработанная.csv сохранён.


In [3]:
import pandas as pd

# Загружаем файлы
words_df = pd.read_csv("обновленная_лексика_мигранты_обработанная.csv")
merged_df = pd.read_excel("merged_with_updated_classes.xlsx")

# Приводим слова к нижнему регистру, убираем пробелы и создаём множество
words_set = set(word.strip().lower() for word in words_df["words"] if isinstance(word, str))

# Добавляем новый столбец в merged_df для фиксации совпадений
merged_df["Совпавший термин"] = ""

# Проверка совпадений
for i, row in merged_df.iterrows():
    matched_word = None

    # Проверяем в "Начальных формах"
    if isinstance(row["Начальные формы"], str) and row["Начальные формы"].strip().lower() in words_set:
        matched_word = row["Начальные формы"].strip()

    # Проверяем в "termin"
    elif isinstance(row["termin"], str) and row["termin"].strip().lower() in words_set:
        matched_word = row["termin"].strip()

    # Проверяем в "Классы глоссариев"
    elif isinstance(row["Классы глоссариев"], str) and row["Классы глоссариев"].strip().lower() in words_set:
        matched_word = row["Классы глоссариев"].strip()

    # Сохраняем совпадение
    if matched_word:
        merged_df.at[i, "Совпавший термин"] = matched_word

# Сохраняем результат
merged_df.to_excel("merged_with_matches_from_migrant_words.xlsx", index=False)
print("✅ Файл merged_with_matches_from_migrant_words.xlsx сохранён.")

✅ Файл merged_with_matches_from_migrant_words.xlsx сохранён.


In [5]:
import pandas as pd

# Загружаем файлы
merged_df = pd.read_excel("merged_with_matches_from_migrant_words.xlsx")
migrant_df = pd.read_csv("обновленная_лексика_мигранты_обработанная.csv")

# Удалим пробелы и приведём к нижнему регистру в обоих файлах
migrant_df["words_normalized"] = migrant_df["words"].astype(str).str.strip().str.lower()
merged_df["Совпавший термин нормализованный"] = merged_df["Совпавший термин"].astype(str).str.strip().str.lower()

# Столбцы, которые надо скопировать (B–AE = 1:31)
migrant_columns_to_add = migrant_df.columns[1:31]

# Создаём копию итогового датафрейма
result_df = merged_df.copy()

# Добавим нужные столбцы в итоговый датафрейм (на всякий случай, если их нет)
for col in migrant_columns_to_add:
    if col not in result_df.columns:
        result_df[col] = None

# Проходим по строкам merged_df и копируем данные, если есть совпадение
for idx, row in merged_df.iterrows():
    norm_term = row["Совпавший термин нормализованный"]
    match_rows = migrant_df[migrant_df["words_normalized"] == norm_term]

    if not match_rows.empty:
        # Берём первую подходящую строку (если вдруг их несколько)
        match_row = match_rows.iloc[0]
        for col in migrant_columns_to_add:
            result_df.at[idx, col] = match_row[col]

# Сохраняем результат
result_df.drop(columns=["Совпавший термин нормализованный"], inplace=True)
result_df.to_excel("merged_with_matches_and_migrant_data.xlsx", index=False)

print("✅ Готово: файл merged_with_matches_and_migrant_data.xlsx сохранён.")

✅ Готово: файл merged_with_matches_and_migrant_data.xlsx сохранён.
